In [6]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [1]:
PROJECT_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project"
DATASET_PATH = "/content/drive/My Drive/Deep_Learning_Assignments/Project/data"
%cd "{PROJECT_PATH}"

[Errno 2] No such file or directory: '/content/drive/My Drive/Deep_Learning_Assignments/Project'
/content


In [2]:
ls -l "{DATASET_PATH}/train" 

ls: cannot access '/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train': No such file or directory


# Convert to jpg

In [0]:
# !sudo apt install imagemagick
# # /content/drive/My Drive/Deep_Learning_Assignments/Project
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/test/"
# !mogrify -format jpg *.jfif
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/healthy_wheat" 
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/leaf_rust"
# !mogrify -format jpg *.jfif  
# %cd "/content/drive/My Drive/Deep_Learning_Assignments/Project/data/train/stem_rust" 
# !mogrify -format jpg *.jfif 

# Fastai

In [0]:
from fastai import * 
from fastai.vision import *

In [5]:
# https://docs.fast.ai/vision.data.html#ImageDataBunch.from_folder
data = ImageDataBunch.from_folder(DATASET_PATH, train="train/", test='test/', valid_pct=0.15,
        ds_tfms=get_transforms(), size = 224, num_workers=4).normalize(imagenet_stats)
# print(data.classes)

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:454: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: ignored

In [0]:
print(data)

ImageDataBunch;

Train: LabelList (746 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
healthy_wheat,healthy_wheat,healthy_wheat,healthy_wheat,healthy_wheat
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data;

Valid: LabelList (131 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
stem_rust,healthy_wheat,leaf_rust,stem_rust,healthy_wheat
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data;

Test: LabelList (610 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: EmptyLabelList
,,,,
Path: /content/drive/My Drive/Deep_Learning_Assignments/Project/data


In [0]:
# data.show_batch(rows=3, figsize=(15, 15)) 

In [0]:
# https://docs.fast.ai/vision.models.html
# resnet18, resnet34, resnet50, resnet101, resnet152
# squeezenet1_0, squeezenet1_1
# densenet121, densenet169, densenet201, densenet161
# vgg16_bn, vgg19_bn
# alexnet

model = cnn_learner(data, models.vgg16_bn, loss_func = nn.CrossEntropyLoss())
# model.unfreeze()

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/checkpoints/densenet201-c1103571.pth


In [0]:
#model.load('fastai-densenet')

In [0]:
# https://github.com/fastai/course-v3/blob/master/nbs/dl1/lesson1-pets.ipynb

model.fit_one_cycle(20, 1e-3)

epoch,train_loss,valid_loss,time
0,1.641094,0.795295,04:26
1,1.224927,0.386751,02:11
2,0.943902,0.284567,02:09
3,0.764602,0.261833,02:08
4,0.678636,0.505434,02:11
5,0.608632,0.929875,02:09
6,0.576324,0.696865,02:07
7,0.527909,0.350423,02:09
8,0.468807,0.341433,02:12
9,0.422669,0.279696,02:10


In [0]:
model.save('fastai-vgg16_bn')

In [0]:
preds_test, y_test = model.get_preds(DatasetType.Test)

In [0]:
import os
test_path = DATASET_PATH+"/test/test"
# test_path = PROJECT_PATH + "/other-data/test/"
# print(model.data.classes)
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir(test_path)  if s.split('.')[1] != 'jfif']})

for i, c in enumerate(model.data.classes):
  # print(preds_test[:,i].shape)
  # print(c)
  submission[c] = preds_test[:,i]
submission = submission[['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']] # Get same order as sample sub
submission.head()

,ID,leaf_rust,stem_rust,healthy_wheat
0,HFV4BK,0.999276,1.966533e-04,0.000527
1,0L5TC8,0.999999,1.560652e-07,0.000001
2,VW34Z3,0.999843,3.597059e-06,0.000153
3,HGSMHR,0.000001,9.999902e-01,0.000009
4,0K9T9T,0.000214,9.997522e-01,0.000034


In [0]:
submission.to_csv('submission.csv', index = False)

In [0]:
# !ls -l "{test_path}" | wc -l
# test_path = PROJECT_PATH + "/other-data/test/"
# len(os.listdir(test_path))
print(submission.shape)

NameError: ignored